# K-Means Algorithm

In this Jupyter Notebook we will focus on K-Means algorithm. K-means is an entry level clusterization algorithm. Clusterization is subdomain of Machine Learning which is all about finding hidden patterns in datasets. K-means is so called unsupervised learning method. In contrast to traditional methods in unsupervised methods we have data points, but we don't have labels. Based on distances between data algorithm finds k groups of points in a given dataset.
 
K menas finds k (iteger e.g 2 or 16 etc.) groups of data in a given dataset and it make it ONLY based on their place in a given space (e.g $R^2$. or $R^3$). Algorithm is quite simple but suprasingly effective. We inicialize  k random points (called centroids) in a given space. Then we find points closest to each centroid and from that points we form a group. Mean value for each group is new centroid. We repeat algorithm as long as our centroids "move" more then some threshold value $\Delta$

Number k depends on a sytuation and problem. For example we want to reduce number of colors in the image in order to compress its size. We find 256 the most important colors and we replace color of each pixel with one of those 256 colors (the closest one to the original pixel value).

## K-means steps

1. Select $k$ random points as initial centroids
2. Count distances between points in dataset and centroids
3. Assign each point to closest centroid
4. Find mean or each group of points and set it as new centroids
5. Check if centroids moved more then some $\Delta$. If no - repeat steps 2-5, if yes - algorithm covered, we found $k$ centroids for given data

## Imports

In [1]:
import sys,os
import numpy as np
import pandas as pd
import multiprocessing as mp
import random
from functools import reduce
import time
import random
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [2]:
def loadData(fileName, labeled=True):
    mnist_data = pd.read_csv(fileName)
    if labeled:
        label = np.array(mnist_data["label"])
        data = np.array(mnist_data.iloc[:, 1:])
        return data, label
    else:
        return np.array(mnist_data.iloc[:, 1:])

# Select $k$ random points as initial centroids

As initial centroids we take k random, unique points from dataset. We take unique points to avoid a situation where there is a centroid without any points.

In [3]:
def init_centroids(data):
    return random.sample(list(data), 10)

# Count distances between points in dataset and centroids

In order to form a group (based on whitch we will find new centroid) we need to find centroid closest each data point in dataset. Depending on Your problem You can use diffrent methods of mesuring the distance between centroids and data points. 

For our problem euclidean distance is fine. To compute it in an effective way we will make use of matrix methods from numpy package, which are much more efficient than loops. 

$A = \begin{bmatrix}
       a & b \\
       c & d \\
       e & f \\
     \end{bmatrix} \space B = \begin{bmatrix}
       g & h \\
     \end{bmatrix}$    
     
$DistanceMatrix = \begin{bmatrix}
                   \sqrt{(a-g)^2 + (b-h)^2} \\
                   \sqrt{(c-g)^2 + (d-h)^2} \\
                   \sqrt{(e-g)^2 + (f-h)^2} \\
                 \end{bmatrix} = \sqrt{\begin{bmatrix}
                                  a^2 + b^2 \\
                                  c^2 + d^2 \\
                                  e^2 + f^2 \\
                                  \end{bmatrix} + \begin{bmatrix}
                                  g^2 + h^2 \\
                                  g^2 + h^2 \\
                                  g^2 + h^2 \\
                                  \end{bmatrix} - 2\begin{bmatrix}
                                  a*g + b*h \\
                                  c*g + d*h \\
                                  e*g + f*h \\
                                  \end{bmatrix}} = \sqrt{(A*A)_{column \space sum} + (B*B)_{column \space sum} - 2*AB^T}$   
                                  
                                  
<br>

For example we have 3 points in dataset matrix $A = \begin{bmatrix}1 & 2 \\ 2 & 3 \\ 3 & 4 \end{bmatrix}$ and 1 point in centroid matrix $B = \begin{bmatrix}0 & 1 \end{bmatrix}$ 

Euklidean distance between points in $A$ and $B$ = $\sqrt{\begin{bmatrix}(1-0)^2 + (2-1)^2 \\(2-0)^2 + (3-1)^2 \\ (3-0)^2 + (4-1)^2 \\ \end{bmatrix}} = \begin{bmatrix}\sqrt{2} \\ \sqrt{8} \\ \sqrt{18} \\ \end{bmatrix} $ 


*$\sqrt{\begin{bmatrix}1 & 2 \\ 2 & 3 \\ 3 & 4 \end{bmatrix}}$ this is not a 100 percent correct mathematical notation but it is intended to give you some idea of what is going on

In [5]:
def form_cluster(centroids, data):
    centroids_indices = range(len(centroids))
    clusters = {x: [] for x in centroids_indices}
    for xi in data:
        max_similarity = -1  # Initialize with a minimum value
        for cj_index in centroids_indices:
            cj = centroids[cj_index]
            similarity = cosine_similarity([xi], [cj])[0][0]
            if similarity > max_similarity:
                closest_centroid_index = cj_index
                max_similarity = similarity
        clusters[closest_centroid_index].append(xi)
    return clusters.values()

# Find new clusters

For each centroid assign closest points from dataset

In [7]:
def move_centroids(clusters):
    new_centroids = []
    for cluster in clusters:
        cluster_array = np.array(cluster)
        if len(cluster_array) > 0:
            mean_vector = np.mean(cluster_array, axis=0)
            new_centroids.append(mean_vector)
    return new_centroids

# Check if clusters changed

To stop algorithm we need to chceck if centroids are still "moving". To do that we count distance between old centroids and new centroids (corresponding centroids lie on the main diagonal of distance matrix) and if all of distances are smaller then $\Delta$ paramter we know that algorithm had covered and we reached final centroids

In [9]:
def repeat_until_convergence(data, clusters, centroids):
    pre_max_diff = 0
    while True:
        old_centroids = centroids
        centroids = move_centroids(clusters)
        clusters = form_cluster(centroids, data)
        differences = map(lambda x, y: np.linalg.norm(x - y), old_centroids, centroids)
        max_diff = max(differences)
        diff_change = abs((max_diff - pre_max_diff) / np.mean([pre_max_diff, max_diff])) * 100
        pre_max_diff = max_diff
        if np.isnan(diff_change):
            print("Stop worker process id: {0}".format(os.getpid()))
            break
    return clusters, centroids

# Summary


We managed to create an effective clustering algorithm. We showed how it works on the example of color reduction. The same algortym can be effectively used for many problems from different areas by changing only the input X, the number of classes k and possibly the method of measuring distances.